In [141]:
import pandas as pd
import numpy as np
import time 
import warnings

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors, linear_model

from sklearn import metrics,preprocessing ,tree
from sklearn.metrics import f1_score,make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(8)
warnings.filterwarnings("ignore")

In [142]:
def timeit(method):
    def timed(*args, **kw):
        ts=time.time()
        result= method(*args, **kw)
        te=time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te-ts)*1000)
        else:
            print ('%r  %2.2f ms' %\
                  (method.__name__,(te-ts)*1000))
        return result
    return timed

In [143]:
file_loc = "E:\\final\\final_dataset.csv"
df=pd.read_csv(file_loc)
df

,ID,TENURE_QUINTILES_0,TENURE_QUINTILES_1,TENURE_QUINTILES_2,TENURE_QUINTILES_3,Jan,Feb,Mar,Apr,May,...,payment_pr1q_mean_diff_days,FUND_VALUE2,Mode_Month,Mode_Quarter,Mode_Half,Mode_Annual,payment_pr1q_sd_pay_amt,payment_pr1q_num_past_due_30d,NO_OF_CHILDREN2,lapse_var
0,1,0,1,0,0,0,0,0,0,1,...,46.854693,1.251947e+07,1,0,0,0,6087.007464,1,0,0
1,2,0,0,1,0,0,0,1,0,0,...,46.215029,3.790000e+07,1,0,0,0,3050.669914,0,0,0
2,3,0,0,1,0,1,0,0,0,0,...,14.498615,1.235961e+07,1,0,0,0,11687.537130,1,0,0
3,4,1,0,0,0,0,0,1,0,0,...,23.824406,6.667302e+06,1,0,0,0,11349.370640,0,0,0
4,5,1,0,0,0,0,0,0,0,0,...,0.931913,1.232416e+07,1,0,0,0,8186.397688,1,0,0
5,6,0,0,1,0,0,0,0,0,0,...,46.490118,2.070000e+07,1,0,0,0,6181.590677,1,1,0
6,7,0,0,1,0,0,0,0,0,0,...,52.431916,7.567418e+06,1,0,0,0,6144.428255,1,2,0
7,8,0,1,0,0,0,0,0,0,1,...,45.878482,2.580000e+07,1,0,0,0,7927.931607,1,3,0
8,9,1,0,0,0,0,0,1,0,0,...,46.053896,1.232036e+07,1,0,0,0,6172.474263,0,0,0
9,10,0,1,0,0,0,0,1,0,0,...,45.030253,4.787733e+06,1,0,0,0,6312.144542,1,0,0


In [144]:
df.shape

(60007, 35)

In [145]:
from sklearn.neighbors import KNeighborsClassifier as dt
clf=dt()

In [146]:
clf

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [147]:
x= df.iloc[:,0:len(df.columns)-1].values
y=df.iloc[:,-1].values

In [148]:
x.shape

(60007, 34)

In [149]:
y.shape

(60007,)

In [150]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [151]:
x_train.shape

(45005, 34)

In [152]:
x_test.shape

(15002, 34)

In [153]:
scores = cross_val_score(clf, x_train, y_train, cv=10, scoring='f1_macro')

In [154]:
scores.mean()

0.48629846254453846

In [155]:
#fit the model
clf.fit(x_train, y_train)
#make prdeiction
train_predictions = clf.predict(x_train)
test_predictions = clf.predict(x_test)

In [156]:
clf

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [157]:
train_cols = df.columns[0:len(df.columns)-1]
target_cols = df.columns[-1]

In [158]:
print ('The training F1 score is: ', f1_score(train_predictions, y_train))
print ('The testing F1 score is : ', f1_score(test_predictions, y_test))
y_test

The training F1 score is:  0.07025316455696201
The testing F1 score is :  0.003913894324853229


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [159]:
# parameters = {'algorithm': ['auto'], 'leaf_size':[30], 'metric':['minkowski'],
#  'metric_params':[None], 'n_jobs':[1], 'n_neighbors':[5], 'p':[2],
#  'weights':['niform']}



# parameters = {'bootstrap': [True],
#   'max_depth': [10,20,30,40, 50, 60,70,80,90,100, None],
#    'max_features': ['auto', 'sqrt'],
#    'min_samples_leaf': [1, 2, 4],
#    'min_samples_split': [2, 5, 10],
#    'n_estimators': [200]}


# {'max_depth': [1,2,3,4,5,6,7,8,9,10],
#           'min_samples_leaf': [1,2,3,4,5],
#           'min_samples_split': [2,3,4,5],
#           'criterion': ['gini','entropy']}
scorer = make_scorer(f1_score)

In [160]:
@timeit
def generate_clf_from_search(grid_or_random, clf, parameters, scorer, x, y):
    if grid_or_random == 'Grid':
        search_obj = GridSearchCV(clf, parameters, scoring = scorer)
    elif grid_or_random == 'Random':
        search_obj = RandomizedSearchCV(clf, parameters, scoring=scorer)
    fit_obj = search_obj.fit(x,y)
    best_clf = fit_obj.best_estimator_
    return best_clf, search_obj, fit_obj

In [161]:
clf

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [162]:
best_clf_grid, search, fit = generate_clf_from_search("Grid",
                                                     clf,
                                                     parameters,
                                                     scorer,
                                                     x_train,
                                                     y_train)

ValueError: weights not recognized: should be 'uniform', 'distance', or a callable function

In [ ]:
fit.cv_results_['params'][0]

In [ ]:
clf

In [ ]:
best_clf_grid

In [ ]:
#fit.cv_results_['mean_test_score'][0]

In [ ]:
#len(fit.cv_results_['mean_test_score'])

In [ ]:
out_f1_values = fit.cv_results_['mean_test_score']

In [ ]:
############################################################################################################################
out_f1_values.sort()
max(out_f1_values)
############################################################################################################################

In [ ]:
plt.plot(out_f1_values)

In [ ]:
r_values = np.random.choice(out_f1_values, 1, replace = False)

In [ ]:
r_values

In [ ]:
prev_score=max(r_values[r_values>0.65])
prev_score